# LME銅 Cash/3Mスプレッド 時系列分析

## 分析目的
このノートブックは、LME銅Cash/3Mスプレッドの時系列特性を詳細に分析し、予測モデリングの基盤を構築します。

### 学習目標:
1. **定常性の理解**: スプレッドデータの統計的性質
2. **自己相関の分析**: 過去の値と現在の値の関係
3. **差分化の効果**: 非定常データを定常化する手法
4. **モデル選択**: ARIMA等のパラメータ決定
5. **予測精度**: 時系列予測の評価方法

### 教科書的アプローチ:
各ステップを小さく分割し、理論と実践を組み合わせて学習していきます。

In [4]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
from scipy import stats

# 時系列分析ライブラリ
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from arch import arch_model
import itertools

warnings.filterwarnings('ignore')
load_dotenv()

# データベース設定
db_config = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'database': os.getenv('DB_NAME', 'lme_copper_db'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'password'),
    'port': os.getenv('DB_PORT', '5432')
}

connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"

# スタイル設定
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = [15, 10]

print("Cash/3M スプレッド時系列分析")
print("="*50)

Cash/3M スプレッド時系列分析


## Step 1: データ読み込みと基本的な可視化

### 理論:
時系列分析の第一歩は、データの基本的な特性を理解することです。
- **トレンド**: 長期的な方向性
- **季節性**: 定期的なパターン
- **不規則変動**: ランダムな変動

In [ ]:
def load_comprehensive_spread_data():
    """
    複数のデータソースからスプレッドデータを読み込み
    """
    print("データ読み込み開始...")
    
    data_sources = [
        {
            'name': 'lme_copper_prices',
            'spread_query': """
                SELECT 
                    trade_date,
                    close_price as spread_value
                FROM lme_copper_prices 
                WHERE ric_code = 'CMCU0-3'
                ORDER BY trade_date
            """,
            'component_query': """
                SELECT 
                    p1.trade_date,
                    p1.close_price as cash_price,
                    p2.close_price as future_3m_price,
                    (p1.close_price - p2.close_price) as spread_value
                FROM 
                    (SELECT trade_date, close_price FROM lme_copper_prices WHERE ric_code = 'CMCU0') p1
                INNER JOIN 
                    (SELECT trade_date, close_price FROM lme_copper_prices WHERE ric_code = 'CMCU3') p2
                    ON p1.trade_date = p2.trade_date
                ORDER BY p1.trade_date
            """
        },
        {
            'name': 'lme_copper_futures',
            'spread_query': """
                SELECT 
                    trade_date,
                    close_price as spread_value
                FROM lme_copper_futures 
                WHERE ric_code = 'CMCU0-3'
                ORDER BY trade_date
            """,
            'component_query': """
                SELECT 
                    p1.trade_date,
                    p1.close_price as cash_price,
                    p2.close_price as future_3m_price,
                    (p1.close_price - p2.close_price) as spread_value
                FROM 
                    (SELECT trade_date, close_price FROM lme_copper_futures WHERE ric_code = 'CMCU0') p1
                INNER JOIN 
                    (SELECT trade_date, close_price FROM lme_copper_futures WHERE ric_code = 'CMCU3') p2
                    ON p1.trade_date = p2.trade_date
                ORDER BY p1.trade_date
            """
        }
    ]
    
    try:
        engine = create_engine(connection_string)
        
        for source in data_sources:
            print(f"\n🔍 {source['name']}からデータ取得中...")
            
            # 直接スプレッドデータを試行
            try:
                df = pd.read_sql(source['spread_query'], engine)
                if df is not None and len(df) > 0:
                    df['trade_date'] = pd.to_datetime(df['trade_date'])
                    df.set_index('trade_date', inplace=True)
                    print(f"✅ 直接スプレッドデータ: {len(df)} レコード")
                    return df
            except Exception as e:
                print(f"   直接スプレッドデータ取得失敗: {e}")
            
            # コンポーネントから計算を試行
            try:
                df = pd.read_sql(source['component_query'], engine)
                if df is not None and len(df) > 0:
                    df['trade_date'] = pd.to_datetime(df['trade_date'])
                    df.set_index('trade_date', inplace=True)
                    print(f"✅ コンポーネント計算: {len(df)} レコード")
                    return df
            except Exception as e:
                print(f"   コンポーネント計算失敗: {e}")
    
    except Exception as e:
        print(f"⚠️ データベース接続失敗: {e}")
    
    # 実データ取得失敗
    print("\n🚨 CRITICAL ERROR: 実データの取得に完全に失敗しました")
    print("\n【データベース診断】")
    print("1. データベース接続: 失敗")
    print("2. テーブル存在確認: 未完了") 
    print("3. データクエリ: 失敗")
    print("\n【解決方法】")
    print("- データベースにCash/3Mスプレッドデータが存在するか確認してください")
    print("- 以下のコマンドでデータを確認:")
    print("  psql -h localhost -U postgres -d lme_copper_db")
    print("  \\dt")
    print("  SELECT COUNT(*) FROM lme_copper_prices WHERE ric_code = 'CMCU0-3';")
    print("- データ収集スクリプトを実行してデータを追加してください")
    print("\n【トラブルシューティング】")
    print("1. PostgreSQLが起動しているか確認: pg_ctl status")
    print("2. 接続設定を確認: .envファイルのDB_HOST, DB_NAME, DB_USER, DB_PASSWORD") 
    print("3. テーブル作成スクリプトが実行されているか確認")
    print("\n分析を続行できません。上記の問題を解決してから再実行してください。")
    
    return None

# データ読み込み実行
df = load_comprehensive_spread_data()
if df is None:
    print("\n実データが利用できないため、これ以降の分析セルは実行できません。")
else:
    print(f"\n📊 最終データセット: {len(df)} レコード")
    print(df.head())

# ACF/PACF分析
if df is not None:
    spread_series = df['spread_value']
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))
    
    # 元データのACF
    plot_acf(spread_series, ax=axes[0,0], lags=40, alpha=0.05)
    axes[0,0].set_title('Autocorrelation Function (ACF) - Original Data', fontsize=12, fontweight='bold')
    axes[0,0].grid(True, alpha=0.3)
    
    # 元データのPACF
    plot_pacf(spread_series, ax=axes[0,1], lags=40, alpha=0.05)
    axes[0,1].set_title('Partial Autocorrelation Function (PACF) - Original Data', fontsize=12, fontweight='bold')
    axes[0,1].grid(True, alpha=0.3)
    
    # 一次差分のACF（定常化のため）
    spread_diff = spread_series.diff().dropna()
    plot_acf(spread_diff, ax=axes[1,0], lags=40, alpha=0.05)
    axes[1,0].set_title('Autocorrelation Function (ACF) - First Difference', fontsize=12, fontweight='bold')
    axes[1,0].grid(True, alpha=0.3)
    
    # 一次差分のPACF
    plot_pacf(spread_diff, ax=axes[1,1], lags=40, alpha=0.05)
    axes[1,1].set_title('Partial Autocorrelation Function (PACF) - First Difference', fontsize=12, fontweight='bold')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 一次差分の定常性検定
    print("\n" + "="*70)
    print("一次差分データの定常性検定")
    print("="*70)
    
    adf_diff = adf_test(spread_diff, "Cash/3M Spread (一次差分)")
    kpss_diff = kpss_test(spread_diff, "Cash/3M Spread (一次差分)")
    
    # 数値的な相関分析
    print(f"\n{'='*50}")
    print("相関分析サマリー")
    print(f"{'='*50}")
    
    # 主要ラグの自己相関
    from statsmodels.tsa.stattools import acf, pacf
    
    acf_values = acf(spread_diff, nlags=10, alpha=0.05)
    pacf_values = pacf(spread_diff, nlags=10, alpha=0.05)
    
    print("主要ラグの自己相関 (一次差分):")
    for i in range(1, 6):
        print(f"ラグ{i}: ACF = {acf_values[0][i]:.3f}, PACF = {pacf_values[0][i]:.3f}")
    
    # Ljung-Box検定（残差の独立性検定）
    lb_test = acorr_ljungbox(spread_diff, lags=10, return_df=True)
    print(f"\nLjung-Box検定 (残差の独立性):")
    print(f"p値 (ラグ10): {lb_test['lb_pvalue'].iloc[-1]:.6f}")
    
    if lb_test['lb_pvalue'].iloc[-1] > 0.05:
        print("✅ 残差は独立（ホワイトノイズ的）")
    else:
        print("❌ 残差に自己相関あり（追加モデリングが必要）")
else:
    print("\n⚠️ データが利用できないため、ACF/PACF分析をスキップします。")
    print("上記のデータ読み込みの問題を解決してから再実行してください。")

In [ ]:
# 定常性検定（データが存在する場合のみ実行）
if df is not None:
    spread_series = df['spread_value']
    
    def adf_test(series, title=""):
        """
        ADF検定（拡張ディッキー・フラー検定）の実行
        H0: 単位根あり（非定常）
        H1: 単位根なし（定常）
        """
        print(f"\n{'='*50}")
        print(f"ADF検定結果: {title}")
        print(f"{'='*50}")
        
        result = adfuller(series, autolag='AIC')
        
        print(f"ADF統計量: {result[0]:.6f}")
        print(f"p値: {result[1]:.6f}")
        print(f"使用ラグ数: {result[2]}")
        print(f"観測数: {result[3]}")
        
        print("\n臨界値:")
        for key, value in result[4].items():
            print(f"\t{key}: {value:.3f}")
        
        # 結果の解釈
        if result[1] <= 0.05:
            print(f"\n✅ 結果: p値 = {result[1]:.6f} < 0.05")
            print("帰無仮説を棄却 → 時系列は定常です")
        else:
            print(f"\n❌ 結果: p値 = {result[1]:.6f} > 0.05")
            print("帰無仮説を棄却できない → 時系列は非定常です")
        
        return result

    def kpss_test(series, title=""):
        """
        KPSS検定の実行
        H0: 定常
        H1: 非定常
        """
        print(f"\n{'='*50}")
        print(f"KPSS検定結果: {title}")
        print(f"{'='*50}")
        
        result = kpss(series, regression='c')
        
        print(f"KPSS統計量: {result[0]:.6f}")
        print(f"p値: {result[1]:.6f}")
        print(f"使用ラグ数: {result[2]}")
        
        print("\n臨界値:")
        for key, value in result[3].items():
            print(f"\t{key}: {value:.3f}")
        
        # 結果の解釈
        if result[1] <= 0.05:
            print(f"\n❌ 結果: p値 = {result[1]:.6f} < 0.05")
            print("帰無仮説を棄却 → 時系列は非定常です")
        else:
            print(f"\n✅ 結果: p値 = {result[1]:.6f} > 0.05")
            print("帰無仮説を棄却できない → 時系列は定常です")
        
        return result

    # ADF検定
    adf_result = adf_test(spread_series, "Cash/3M Spread (元データ)")
    
    # KPSS検定
    kpss_result = kpss_test(spread_series, "Cash/3M Spread (元データ)")
    
    print(f"\n{'='*70}")
    print("定常性検定まとめ")
    print(f"{'='*70}")
    print("検定結果:")
    print(f"ADF検定: {'定常' if adf_result[1] <= 0.05 else '非定常'}")
    print(f"KPSS検定: {'定常' if kpss_result[1] > 0.05 else '非定常'}")
    
    if adf_result[1] <= 0.05 and kpss_result[1] > 0.05:
        print("\n✅ 総合判定: 時系列は定常です")
    elif adf_result[1] > 0.05 and kpss_result[1] <= 0.05:
        print("\n❌ 総合判定: 時系列は非定常です")
    else:
        print("\n⚠️ 総合判定: 検定結果が矛盾しています。追加検証が必要です")
else:
    print("\n⚠️ データが利用できないため、定常性検定をスキップします。")
    print("上記のデータ読み込みの問題を解決してから再実行してください。")

## Step 3: 自己相関関数 (ACF) と偏自己相関関数 (PACF) の分析

### 理論:
**自己相関関数 (ACF):**
- 時系列と自分自身のラグ版との相関
- MA(q)モデルのqを決定するのに使用

**偏自己相関関数 (PACF):**
- 中間のラグの影響を除いた直接的な相関
- AR(p)モデルのpを決定するのに使用

**パターン識別:**
- AR(p): PACF がp次で急激に減衰、ACF は指数的減衰
- MA(q): ACF がq次で急激に減衰、PACF は指数的減衰
- ARMA(p,q): 両方とも指数的減衰

In [ ]:
# ACF/PACF分析
if df is not None:
    spread_series = df['spread_value']
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 12))
    
    # 元データのACF
    plot_acf(spread_series, ax=axes[0,0], lags=40, alpha=0.05)
    axes[0,0].set_title('自己相関関数 (ACF) - 元データ', fontsize=12, fontweight='bold')
    axes[0,0].grid(True, alpha=0.3)
    
    # 元データのPACF
    plot_pacf(spread_series, ax=axes[0,1], lags=40, alpha=0.05)
    axes[0,1].set_title('偏自己相関関数 (PACF) - 元データ', fontsize=12, fontweight='bold')
    axes[0,1].grid(True, alpha=0.3)
    
    # 一次差分のACF（定常化のため）
    spread_diff = spread_series.diff().dropna()
    plot_acf(spread_diff, ax=axes[1,0], lags=40, alpha=0.05)
    axes[1,0].set_title('自己相関関数 (ACF) - 一次差分', fontsize=12, fontweight='bold')
    axes[1,0].grid(True, alpha=0.3)
    
    # 一次差分のPACF
    plot_pacf(spread_diff, ax=axes[1,1], lags=40, alpha=0.05)
    axes[1,1].set_title('偏自己相関関数 (PACF) - 一次差分', fontsize=12, fontweight='bold')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 一次差分の定常性検定
    print("\n" + "="*70)
    print("一次差分データの定常性検定")
    print("="*70)
    
    adf_diff = adf_test(spread_diff, "Cash/3M Spread (一次差分)")
    kpss_diff = kpss_test(spread_diff, "Cash/3M Spread (一次差分)")
    
    # 数値的な相関分析
    print(f"\n{'='*50}")
    print("相関分析サマリー")
    print(f"{'='*50}")
    
    # 主要ラグの自己相関
    from statsmodels.tsa.stattools import acf, pacf
    
    acf_values = acf(spread_diff, nlags=10, alpha=0.05)
    pacf_values = pacf(spread_diff, nlags=10, alpha=0.05)
    
    print("主要ラグの自己相関 (一次差分):")
    for i in range(1, 6):
        print(f"ラグ{i}: ACF = {acf_values[0][i]:.3f}, PACF = {pacf_values[0][i]:.3f}")
    
    # Ljung-Box検定（残差の独立性検定）
    lb_test = acorr_ljungbox(spread_diff, lags=10, return_df=True)
    print(f"\nLjung-Box検定 (残差の独立性):")
    print(f"p値 (ラグ10): {lb_test['lb_pvalue'].iloc[-1]:.6f}")
    
    if lb_test['lb_pvalue'].iloc[-1] > 0.05:
        print("✅ 残差は独立（ホワイトノイズ的）")
    else:
        print("❌ 残差に自己相関あり（追加モデリングが必要）")
else:
    print("\n⚠️ データが利用できないため、ACF/PACF分析をスキップします。")
    print("上記のデータ読み込みの問題を解決してから再実行してください。")

## Step 4: ARIMA モデルの選択と推定

### 理論:
**ARIMA(p,d,q) モデル:**
- p: 自己回帰項の次数 (AR)
- d: 差分の次数 (I: Integrated)
- q: 移動平均項の次数 (MA)

**モデル選択基準:**
- **AIC (赤池情報量基準)**: 小さいほど良い
- **BIC (ベイズ情報量基準)**: 小さいほど良い
- **残差診断**: ホワイトノイズ性の確認

In [ ]:
        # Q-Qプロット
        from scipy.stats import probplot
        probplot(residuals, dist="norm", plot=axes[1,0])
        axes[1,0].set_title('Q-Q Plot (Normality Test)', fontsize=12, fontweight='bold')
        axes[1,0].grid(True, alpha=0.3)

## Step 5: 予測と評価

### 理論:
**予測評価指標:**
- **MAE**: 平均絶対誤差
- **RMSE**: 二乗平均平方根誤差  
- **MAPE**: 平均絶対パーセント誤差
- **方向性精度**: 上昇/下降の予測的中率

**時系列クロスバリデーション:**
- 時系列の順序を保持した分割
- ウォークフォワード分析

In [ ]:
# 予測と評価
if spread_data is not None and best_model is not None:
    # データ分割（最後の30日をテスト用）
    test_size = 30
    train_data = spread_series[:-test_size]
    test_data = spread_series[-test_size:]
    
    print(f"学習データ: {len(train_data)}日")
    print(f"テストデータ: {len(test_data)}日")
    
    # モデル再学習（学習データのみ）
    train_model = ARIMA(train_data, order=best_order_aic)
    fitted_train_model = train_model.fit()
    
    # 予測実行
    forecast_result = fitted_train_model.forecast(steps=test_size, alpha=0.05)
    forecast_values = forecast_result
    
    # 信頼区間取得
    forecast_ci = fitted_train_model.get_forecast(steps=test_size, alpha=0.05)
    conf_int = forecast_ci.conf_int()
    
    # 予測評価指標計算
    mae = np.mean(np.abs(test_data - forecast_values))
    rmse = np.sqrt(np.mean((test_data - forecast_values)**2))
    mape = np.mean(np.abs((test_data - forecast_values) / test_data)) * 100
    
    # 方向性精度
    actual_direction = np.sign(test_data.diff().dropna())
    predicted_direction = np.sign(pd.Series(forecast_values, index=test_data.index).diff().dropna())
    directional_accuracy = (actual_direction == predicted_direction).mean()
    
    print(f"\n{'='*50}")
    print("予測精度評価")
    print(f"{'='*50}")
    print(f"MAE (平均絶対誤差): {mae:.3f}")
    print(f"RMSE (二乗平均平方根誤差): {rmse:.3f}")
    print(f"MAPE (平均絶対パーセント誤差): {mape:.3f}%")
    print(f"方向性精度: {directional_accuracy:.3f} ({directional_accuracy*100:.1f}%)")
    
    # 予測結果の可視化
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    # 予測vs実績
    ax1.plot(train_data.index[-60:], train_data[-60:], label='学習データ', color='blue', alpha=0.7)
    ax1.plot(test_data.index, test_data, label='実際の値', color='green', linewidth=2)
    ax1.plot(test_data.index, forecast_values, label='予測値', color='red', linewidth=2, linestyle='--')
    
    # 信頼区間
    ax1.fill_between(test_data.index, 
                     conf_int.iloc[:, 0], 
                     conf_int.iloc[:, 1], 
                     color='red', alpha=0.2, label='95%信頼区間')
    
    ax1.axhline(y=0, color='black', linestyle='-', alpha=0.5)
    ax1.set_title(f'ARIMA{best_order_aic} 予測結果', fontsize=14, fontweight='bold')
    ax1.set_ylabel('スプレッド (USD/tonne)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 予測誤差
    prediction_errors = test_data - forecast_values
    ax2.plot(test_data.index, prediction_errors, marker='o', linestyle='-', color='red')
    ax2.axhline(y=0, color='black', linestyle='--', alpha=0.7)
    ax2.set_title('予測誤差', fontsize=14, fontweight='bold')
    ax2.set_ylabel('誤差 (USD/tonne)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 誤差統計
    print(f"\n予測誤差統計:")
    print(f"誤差の平均: {prediction_errors.mean():.3f}")
    print(f"誤差の標準偏差: {prediction_errors.std():.3f}")
    print(f"最大正誤差: {prediction_errors.max():.3f}")
    print(f"最大負誤差: {prediction_errors.min():.3f}")
    
    # 信頼区間カバレッジ
    coverage = ((test_data >= conf_int.iloc[:, 0]) & 
                (test_data <= conf_int.iloc[:, 1])).mean()
    print(f"95%信頼区間カバレッジ: {coverage:.3f} ({coverage*100:.1f}%)")

## Step 6: 結果の解釈とまとめ

### 時系列分析から得られた知見の整理
分析結果をトレーディング戦略に活用するための総合的な解釈を行います。

In [ ]:
# 定常化データをDataFrameに保存（次のステップで使用）
if spread_data is not None:
    # 分析結果のまとめ
    stationary_data = pd.DataFrame(index=spread_data.index)
    stationary_data['original_spread'] = spread_data['spread_price']
    stationary_data['diff_spread'] = spread_data['spread_price'].diff()
    
    # 移動統計（ローリング平均・標準偏差）
    stationary_data['rolling_mean_20'] = spread_data['spread_price'].rolling(20).mean()
    stationary_data['rolling_std_20'] = spread_data['spread_price'].rolling(20).std()
    
    # Z-score（標準化）
    stationary_data['zscore'] = (spread_data['spread_price'] - stationary_data['rolling_mean_20']) / stationary_data['rolling_std_20']
    
    # 欠損値除去
    stationary_data = stationary_data.dropna()
    
    print("定常化データ（stationary_data）を作成しました")
    print(f"データ期間: {stationary_data.index.min().date()} to {stationary_data.index.max().date()}")
    print(f"データ数: {len(stationary_data)}")
    
    # 最新10日間のデータ表示
    print("\n最新データ（最後の10日間）:")
    print(stationary_data.tail(10).round(3))
    
    print(f"\n{'='*70}")
    print("時系列分析 総合まとめ")
    print(f"{'='*70}")
    
    print("\n【定常性について】")
    if adf_result[1] <= 0.05:
        print("✅ 元データは定常的 → 差分化は不要")
        print("   → レベル値での予測が可能")
    else:
        print("❌ 元データは非定常 → 差分化が必要")
        print("   → 変化量での予測が適している")
    
    print("\n【モデリング結果】")
    if best_model is not None:
        print(f"最適モデル: ARIMA{best_order_aic}")
        print(f"AIC: {best_aic:.2f}")
        print(f"予測精度 (MAE): {mae:.3f}")
        print(f"方向性精度: {directional_accuracy*100:.1f}%")
    
    print("\n【トレーディングへの示唆】")
    
    # 現在のスプレッド位置
    current_spread = stationary_data['original_spread'].iloc[-1]
    current_zscore = stationary_data['zscore'].iloc[-1]
    
    print(f"現在のスプレッド: ${current_spread:.1f}")
    print(f"現在のZ-score: {current_zscore:.2f}")
    
    if abs(current_zscore) > 2:
        print("⚠️ 極端な水準 → 逆張り戦略を検討")
        if current_zscore > 2:
            print("   → 強いバックワーデーション（売り検討）")
        else:
            print("   → 強いコンタンゴ（買い検討）")
    elif abs(current_zscore) > 1:
        print("📊 やや極端な水準 → 注意深く監視")
    else:
        print("✅ 正常範囲 → トレンドフォロー戦略")
    
    # 予測精度に基づく信頼性
    if directional_accuracy > 0.6:
        print(f"\n✅ 予測精度が高い（{directional_accuracy*100:.1f}%）→ モデル信号を重視")
    elif directional_accuracy > 0.5:
        print(f"\n📊 予測精度は中程度（{directional_accuracy*100:.1f}%）→ 他の指標と組み合わせ")
    else:
        print(f"\n❌ 予測精度が低い（{directional_accuracy*100:.1f}%）→ モデル改善が必要")
    
    print(f"\n{'='*70}")
    print("次のステップ: 機械学習モデルとの比較・アンサンブル")
    print(f"{'='*70}")